In [1]:
from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, UpSampling2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.callbacks import Callback
import random
import glob
import wandb
from wandb.keras import WandbCallback

C:\Users\hohsiny\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import subprocess
import os
from PIL import Image
import numpy as np
import cv2

run = wandb.init()
config = run.config

config.num_epochs = 1
config.batch_size = 4
config.img_dir = "images"
config.height = 256
config.width = 256

val_dir = 'test'
train_dir = 'train'

wandb: ERROR: Headless mode isn't supported on Windows. If you want to use W&B, please use "wandb run ..."; running normally.


In [6]:
# automatically get the data if it doesn't exist
if not os.path.exists("train"):
    print("Downloading flower dataset...")
    subprocess.check_output("curl https://storage.googleapis.com/l2kzone/flowers.tar | tar xz", shell=True)

def my_generator(batch_size, img_dir):
    """A generator that returns black and white images and color images"""
    image_filenames = glob.glob(img_dir + "\*")
    counter = 0
    while True:
        bw_images = np.zeros((batch_size, config.width, config.height))
        color_images = np.zeros((batch_size, config.width, config.height, 3))
        random.shuffle(image_filenames) 
        if ((counter+1)*batch_size>=len(image_filenames)):
              counter = 0
        for i in range(batch_size):
              img = Image.open(image_filenames[counter + i]).resize((config.width, config.height))
              color_images[i] = np.array(img)
              bw_images[i] = np.array(img.convert('L'))
        yield (bw_images, color_images)
        counter += batch_size 

In [7]:
(test_bw_images, test_color_images) = next(my_generator(145, train_dir))

In [9]:
test_bw_images.shape

(145, 256, 256)

In [10]:
test_color_images.shape

(145, 256, 256, 3)